In [25]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import display
from loguru import logger

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

filename_process_data = '20231030_solarMED.csv'
filename_process_data2 = '20231030_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

sample_rate = '60s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Load variables information
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Load plot configuration
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(data_path / filename_process_data, parse_dates=True, index_col='TimeStamp')
# Rename index column to "time"
df.index.names = ['time']
# Set UTC timezone
df = df.tz_localize('UTC')

# TEMP: Read additional MED data from librescada logging, in the future everything should be read directly from librescada
df_aux = pd.read_csv(data_path / filename_process_data2, parse_dates=True, index_col='time')
# Set UTC timezone
df_aux = df_aux.tz_localize('UTC')
# Add columns from df_aux to df
df = pd.concat([df, df_aux], axis=1)

display(df.head())

,FT-AQU-100,FT-AQU-101,TT-AQU-106,TT-AQU-107a,TT-AQU-109,CT-DES-001,FT-DES-003,FT-DES-005,SW2TC1,TE-DES-015,...,TT-SF-007,TT-SF-008,TT-SF-009,TT-SF-010,vfd_prod_power,PK-MED-E01-pa,vfd_brine_power,HW1TT21,vfd_sw_power,FT-DES-002_VFD
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:01+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:02+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:03+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-30 07:30:04+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Trim the dataframe to the desired time range

# df = df[:-19*60] # Time strings can be used, check past code
# Remove until 10:30

# df = df[initial_datetime:final_datetime]
# logger.info(f'Trimmed dataframe from {df.index[0]} to {df.index[-1]}')

df_copy = df.copy()

In [28]:
# Preprocessing
from phd_visualizations.utils import rename_signal_ids_to_var_ids

%autoreload 2

# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

display(df.head())

,qmed_s,qts_dis,Tts_h_out,Tmed_s_in,Tts_c_b_in,wmed_f,qmed_f,qmed_d,Tmed_c_out,Tmed_c_in,...,Thx_p_out,Thx_s_in,Thx_s_out,qhx_p,qhx_s,T3wv_src,T3wv_dis_in,T3wv_dis_out,q3wv_src,q3wv_dis
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:31:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:32:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:33:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681
2023-10-30 07:34:00+00:00,0.008681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.008681


In [29]:
# Calculate power consumptions from fitted curves

# from calculations import power_consumption
# 
# df["Ce_dc"] = power_consumption(df["w_dc"].to_numpy(), actuator='fan_dc')
# df["Ce_wct"] = power_consumption(df["w_wct"].to_numpy(), actuator='fan_wct')
# df["Ce_c"] = power_consumption(df["q_c"].to_numpy(), actuator='pump_c')

# TODO: Add all missing consumptions, if not measured directly, calculate from fitted curves
df["Ce"] = df['Jmed_b'] + df['Jmed_c'] + df['Jmed_d'] + df['Jmed_s_f']*1e-3 # kW

In [30]:
# Calculate Benefits

df["B"] = (cost_w * df["qmed_d"] - cost_e * df["Ce"] ) * sample_rate_numeric/3600 # u.m.
df["sf_active"] = df["qsf"] > 0.1
df["med_active"] = df["qmed_f"] > 0.5

In [31]:
df_copy = df.copy()

In [32]:
# Convert units to model units
from phd_visualizations.utils.units import unit_conversion

%autoreload 2

df_copy = df.copy()

df_copy = unit_conversion(df_copy, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

display(df_copy.head())

2024-02-03 21:43:29.786 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tamb to C from C
2024-02-03 21:43:29.789 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tmed_c_in to C from C
2024-02-03 21:43:29.794 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tmed_s_in to C from C
2024-02-03 21:43:29.801 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tmed_s_out to C from C
2024-02-03 21:43:29.802 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tmed_c_out to C from C
2024-02-03 21:43:29.803 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tsf_in to C from C
2024-02-03 21:43:29.803 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tsf_out to C from C
2024-02-03 21:43:29.804 | DEBUG    | phd_visualizations.utils.units:unit_conversion:521 - Updated Tts_h_t_in to C from C
2024-02-03 21:43:29.805 | DEBUG    | phd_visual

,qmed_s,qts_dis,Tts_h_out,Tmed_s_in,Tts_c_b_in,wmed_f,qmed_f,qmed_d,Tmed_c_out,Tmed_c_in,...,qhx_s,T3wv_src,T3wv_dis_in,T3wv_dis_out,q3wv_src,q3wv_dis,Ce,B,sf_active,med_active
time,,,,,,,,,,,,,,,,,,,,,
2023-10-30 07:30:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,0.0,NaN,NaN,NaN,NaN,0.03125,NaN,NaN,False,False
2023-10-30 07:31:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,0.0,NaN,NaN,NaN,NaN,0.03125,NaN,NaN,False,False
2023-10-30 07:32:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,0.0,NaN,NaN,NaN,NaN,0.03125,NaN,NaN,False,False
2023-10-30 07:33:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,0.0,NaN,NaN,NaN,NaN,0.03125,NaN,NaN,False,False
2023-10-30 07:34:00+00:00,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,...,0.0,NaN,NaN,NaN,NaN,0.03125,NaN,NaN,False,False


In [33]:
 # Print some prior and after for comparison: qmed_s, qsf, qhx_p, qhx_s, qts_src, qts_dis, Jmed_s_f, wmed_f
# Value at 11:45:00

values_ant = df.loc['2023-10-30 11:45:00']
values_post = df_copy.loc['2023-10-30 11:45:00']
var_ids = ['qmed_s', 'qsf', 'qhx_p', 'qhx_s', 'qts_src', 'qts_dis', 'Jmed_s_f', 'wmed_f']
# Prior
for var_id in var_ids:
    logger.info(f'{var_id}: {values_ant[var_id]:.2f} {vars_config[var_id]["units_scada"]} -> {values_post[var_id]:.2f} {vars_config[var_id]["units_model"]}')


2024-02-03 21:43:29.905 | INFO     | __main__:<module>:9 - qmed_s: 12.00 L/s -> 43.18 m3/h
2024-02-03 21:43:29.905 | INFO     | __main__:<module>:9 - qsf: 132.39 L/min -> 2.21 m3/h
2024-02-03 21:43:29.906 | INFO     | __main__:<module>:9 - qhx_p: 132.39 L/min -> 2.21 m3/h
2024-02-03 21:43:29.906 | INFO     | __main__:<module>:9 - qhx_s: 23.87 L/min -> 0.40 m3/h
2024-02-03 21:43:29.906 | INFO     | __main__:<module>:9 - qts_src: 23.87 L/min -> 0.40 m3/h
2024-02-03 21:43:29.906 | INFO     | __main__:<module>:9 - qts_dis: 5.01 L/s -> 18.04 m3/h
2024-02-03 21:43:29.907 | INFO     | __main__:<module>:9 - Jmed_s_f: 8499.60 W -> 8.50 kW
2024-02-03 21:43:29.907 | INFO     | __main__:<module>:9 - wmed_f: 0.78 mS/cm -> 0.78 g/kg


In [34]:
# If satisfied with unit conversion, overwrite df with df_copy
df = df_copy.copy()
logger.debug(list(df.columns))

2024-02-03 21:43:30.108 | DEBUG    | __main__:<module>:3 - ['qmed_s', 'qts_dis', 'Tts_h_out', 'Tmed_s_in', 'Tts_c_b_in', 'wmed_f', 'qmed_f', 'qmed_d', 'Tmed_c_out', 'Tmed_c_in', 'Tamb', 'qts_src', 'qsf', 'I', 'Tts_c_t', 'Tts_c_m', 'Tts_c_b', 'Tts_h_t', 'Tts_h_m', 'Tts_h_b', 'Tts_c_b_out', 'Tts_h_t_in', 'Tsf_in', 'Tsf_out', 'Jmed_d', 'Jmed_s_f', 'Jmed_b', 'Tmed_s_out', 'Jmed_c', 'qmed_c', 'Tts_src_in', 'Tts_dis', 'Tts_dis_in', 'Thx_p_in', 'Thx_p_out', 'Thx_s_in', 'Thx_s_out', 'qhx_p', 'qhx_s', 'T3wv_src', 'T3wv_dis_in', 'T3wv_dis_out', 'q3wv_src', 'q3wv_dis', 'Ce', 'B', 'sf_active', 'med_active']


In [41]:
from iapws import IAPWS97 as w_props

def calculate_powers(row, max_power: float = 250, min_power=0):
    try:
        # Thermal storage input power
        w_p = w_props(P=0.16, T=(row["Thx_s_out"]+row["Thx_s_in"])/2+273.15) # MPa, K
        row["Pts_src"] = row["qts_src"]/3600 * w_p.rho * w_p.cp * (row["Thx_s_out"]-row["Thx_s_in"]) # kW
        row["Pts_src"] = min(row["Pts_src"], max_power)
        row["Pts_src"] = max(row["Pts_src"], min_power)
        
        # Thermal storage output power
        w_p = w_props(P=0.16, T=(row["Tts_h_out"]+row["Tts_c_b_in"])/2+273.15) # MPa, K
        row["Pts_dis"] = row["qts_dis"]/3600 * w_p.rho * w_p.cp * (row["Tts_h_out"]-row["Tts_c_b_in"]) # kW
        row["Pts_dis"] = min(row["Pts_dis"], max_power)
        row["Pts_dis"] = max(row["Pts_dis"], min_power)

        # Solar field
        w_p = w_props(P=0.16, T=(row["Tsf_in"]+row["Tsf_out"])/2+273.15) # MPa, K
        row["Psf"] = row["qsf"]/3600 * w_p.rho * w_p.cp * (row["Tsf_out"]-row["Tsf_in"]) # kW
        row["Psf"] = min(row["Psf"], max_power)
        row["Psf"] = max(row["Psf"], min_power)

    except Exception as e:
        logger.error(f'Error: {e}')
        row["Pts_src"] = np.nan
        row["Pts_dis"] = np.nan
        row["Psf"] = np.nan

    return row

df = df.apply(calculate_powers, axis=1)

# How many nans?
logger.debug(f'non value Pts_src: {df["Pts_src"].isna().sum()} / {len(df)}')

2024-02-03 21:46:47.785 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.785 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.786 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.786 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.786 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.787 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.787 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.788 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.788 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.788 | ERROR    | __main__:calculate_powers:24 - Error: Incoming out of bound
2024-02-03 21:46:47.789 | ERRO

In [17]:
from phd_visualizations.test_timeseries import experimental_results_plot

%autoreload 2

# Load plot configuration here as well to not have to go back and forth
with open(Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, )

if resample_figures:
    fig.show_dash(
        'inline', 
        config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
    )
else:
    fig.show(
        config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
    )

2024-02-02 16:19:17.652 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:182 - Optimization updates not shown in plot, show_optimization_updates: false
2024-02-02 16:19:17.653 | DEBUG    | phd_visualizations.test_timeseries:add_trace:58 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-02-02 16:19:17.656 | INFO     | phd_visualizations.test_timeseries:add_trace:114 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False
2024-02-02 16:19:17.658 | DEBUG    | phd_visualizations.test_timeseries:add_trace:58 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-02-02 16:19:17.661 | INFO     | phd_visualizations.test_timeseries:add_trace:114 - Trace T<sub>med,c,in</sub> (⁰C) added in yaxis1 (left), row 1, uncertainty: False
2024-02-02 16:19:17.662 | DEBUG    | phd_visualizations.test_timeseries:experimental_results_plot:393 - Adding trace I to right axis 2
2024-02-02 16:19:17.666 | INFO     | phd

In [10]:
# Save figure
save_figure(
    figure_name=f"experimental_results_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=results_path,
    fig=fig, formats=('png', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-01-31 18:57:43.420 | INFO     | phd_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/EURECAT/results/experimental_results_20231030.png
2024-01-31 18:57:43.518 | INFO     | phd_visualizations:save_figure:24 - Figure saved in /home/patomareao/Nextcloud/Juanmi_MED_PSA/EURECAT/results/experimental_results_20231030.html


## Others

In [16]:
# yaxis shifting to better visualize multiple variables, WIP
# Once completed, integrate into experimental_results_plot

import plotly.graph_objects as go

nticks = 5
n_decimal_places = 2

# Create a simple figure with two traces
fig = go.Figure()

ys = [df["Pts_dis"].to_numpy(), df["I"].to_numpy(), df["Tsf_out"].to_numpy()]

# Filter out NaNs

ys = [y[~np.isnan(y)] for y in ys]

# Add first trace
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=ys[0],
        name="P<sub>ts,dis</sub>"
    )
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["Pts_src"], name="P<sub>ts,src</sub>",)
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["Psf"], name="P<sub>sf</sub>",)
)

# Add second trace
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=ys[1],
        name="I (W/m<sup>2</sup>)",
        yaxis="y2"
    )
)

# Add third trace
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=ys[2],
        name="T<sub>sf,out</sub>",
        yaxis="y3"
    )
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["Tsf_in"], name="T<sub>sf,in</sub>", yaxis="y3")
)

spans = [y.max() - y.min() for y in ys]
paddings = [span * 0.1 for span in spans]
ylim0s = [(y.min() - padding, y.max() + padding) for y, padding in zip(ys, paddings)]
tick0s = [ylim0[0] for ylim0 in ylim0s]

# Create new limits based on criteria
ylims = [None for _ in range(len(ys))]
ylims[0] = (ylim0s[0][0], ylim0s[0][1] + spans[0])
ylims[1] = (ylim0s[1][0], ylim0s[1][1] + spans[1])
ylims[2] = (ylim0s[2][0] - spans[2], ylim0s[2][1])

# padding = span * 0.1
# ylim0 = (y.min() - padding, y.max() + padding)
# tick0 = ylim0[0]
# ylim = (ylim0[0] - span, ylim0[1])
# 
# span2 = y2.max() - y2.min()
# padding2 = span2 * 0.1
# ylim0_2 = (y2.min() - padding2, y2.max() + padding2)
# tick0_2 = ylim0_2[0]
# ylim2 = (ylim0_2[0], ylim0_2[1] + span2)
# 
# Create a second y-axis that is positioned at 0.8 (80% of the way across the plot)
fig.update_layout(
    yaxis=dict(
        title_text="kW<sub>th</sub> ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ",
        range=ylims[0],
        tickmode='array',
        # tick0 = tick0s[0],
        # dtick = spans[0]/(nticks-1),
        tickvals=np.round( np.linspace(tick0s[0], tick0s[0] + spans[0], nticks), n_decimal_places ),
        zeroline=False,
        # ticksuffix=" -"
    ),

    yaxis2=dict(
        title_text="W/m<sup>2</sup>‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ",
        range=ylims[1],
        tickvals=np.round( np.linspace(tick0s[1], tick0s[1] + spans[1], nticks), n_decimal_places ),
        overlaying="y",
        side="right",
        zeroline=False,
        # tickprefix="- "
    ),

    yaxis3=dict(
        # domain=[0, 0.1],
        title_text="‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎    ºC",
        
        range=ylims[2],
        tickvals=np.round( np.linspace(tick0s[2], tick0s[2] + spans[2], nticks), n_decimal_places ),
        overlaying="y",
        side="right",
        zeroline=False,
        position = 0.95,
        # show ticks line
        showline=True,
        # tickprefix="- "
    ),
    
    xaxis=dict(
        domain=[0.01, 0.9]
    ),
    
    showlegend=False,
)

fig.show()

In [46]:
import plotly.graph_objects as go
import numpy as np
from phd_visualizations.constants import color_palette

# Visualize the difference between two traces
# https://stackoverflow.com/a/75593357

# Generate data
# x = np.linspace(0, 10, 100)
# trace1 = np.sin(x)
# trace2 = np.cos(x)

df2 = pd.DataFrame({
    # "x": x,
    "Psf": df["Psf"],
    "Pts_dis": df["Pts_dis"],
})
x = df.index
# There is a multiyear bug in Plotly that you cannot use connectgaps and fill in the same plot.
# This is why we set the indicator value to the price value when we do not want to plot the area,
# as this will fill area with the size of 0
# https://github.com/plotly/plotly.js/issues/1132#issuecomment-531030346
df2["trace_pos"] = df2.apply(lambda row: row["Psf"] if row["Psf"] > row["Pts_dis"] else row["Pts_dis"], axis="columns")
df2["trace_neg"] = df2.apply(lambda row: row["Psf"] if row["Psf"] <= row["Pts_dis"] else row["Pts_dis"], axis="columns")


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x,
        y=df2["Pts_dis"],
        name="trace2",
        line_color=color_palette['wct_purple'],
        line_width=3,
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df2["trace_pos"],
        name="positive difference",
        line_color="green",
        connectgaps=False,
        fillcolor="green",
        fill='tonexty',
    )
)

# We need to use an invisible trace so we can reset "next y"
# for the red area indicator
fig.add_trace(
    go.Scatter(
        x=x,
        y=df2["Pts_dis"],
        line_color="rgba(0,0,0,0)",
    )
)
#
fig.add_trace(
    go.Scatter(
        x=x,
        y=df2["trace_neg"],
        name="negative diifference",
        line_color="red",
        connectgaps=False,
        fillcolor="red",
        fill='tonexty',
    )
)

# Show plot
fig.show()


In [ ]:
from models_psa_temp.thermal_storage import calculate_stored_energy

Tmed_s_in_max: float = 90 # ºC, maximum temperature of the hot water inlet, changes dynamically with Tts_h_t
Tmed_s_in_min: float = 60 # ºC, minimum temperature of the hot water inlet, operational limit

Vts_h = [5.94771006, 4.87661781, 2.19737023] # Volume of each control volume of the hot tank (m³)
Vts_c = [5.33410037, 7.56470594, 0.90547187] # Volume of each control volume of the cold tank (m³)

# For the temperatures, vertically stack them
Tts_h = np.stack([df["Tts_h_t"].to_numpy(), df["Tts_h_m"].to_numpy(), df["Tts_h_b"].to_numpy()])
Tts_c = np.stack([df["Tts_c_t"].to_numpy(), df["Tts_c_m"].to_numpy(), df["Tts_c_b"].to_numpy()])

# Calculate stored energy using as reference: Tmed_s_in_min, Tmed_s_in, Tmed_s_in_max
Ets_h = np.zeros( (3, len(df)) ); Ets_c = np.zeros( (3, len(df)) )

for idx, Tmed_s_in in enumerate(df["Tmed_s_in"]):
    try:
        if np.isnan(Tts_h[:,idx]).any() or np.isnan(Tmed_s_in):
            Ets_h[:, idx] = np.nan
        else:
            Ets_h[0, idx] = calculate_stored_energy(Tts_h[:, idx], Vts_h, Tmed_s_in_min)
            if Tmed_s_in < Tmed_s_in_min:
                Ets_h[1, idx] = np.nan
            else:
                Ets_h[1, idx] = calculate_stored_energy(Tts_h[:, idx], Vts_h, Tmed_s_in)            
            Ets_h[2, idx] = calculate_stored_energy(Tts_h[:, idx], Vts_h, Tmed_s_in_max)
    except:
        logger.error(f'Error at index {idx} for Ets_h')
        Ets_h[:, idx] = np.nan
        
    try:
        if np.isnan(Tts_c[:,idx]).any() or np.isnan(Tmed_s_in):
            Ets_c[idx] = np.nan
        else:
            Ets_c[0, idx] = calculate_stored_energy(Tts_c[:, idx], Vts_c, Tmed_s_in_min)
            if Tmed_s_in < Tmed_s_in_min:
                Ets_c[1, idx] = np.nan
            else:
                Ets_c[1, idx] = calculate_stored_energy(Tts_c[:, idx], Vts_c, Tmed_s_in)
            Ets_c[2, idx] = calculate_stored_energy(Tts_c[:, idx], Vts_c, Tmed_s_in_max)
    except:
        logger.error(f'Error at index {idx} for Ets_c')
        Ets_c[:, idx] = np.nan


In [ ]:
# Thermal storage visualization, WIP
# Once completed, move to PhD_visualizations

from phd_visualizations.constants import color_palette


fig = go.Figure()

# Lower temperature
fig.add_trace(go.Scatter(
    x=df.index, y=Ets_h[0,:]+Ets_c[0,:],
    name='Tmed_s_in_min',
    # hoverinfo='x+y',
    mode='lines',
    line=dict(width=1, color=color_palette['c_blue']),
    # stackgroup='Eth'
    fill='tozeroy',
))

# Current temperature
fig.add_trace(go.Scatter(
    x=df.index, y=Ets_h[1,:]+Ets_c[1,:],
    name='Tmed_s_in',
    # hoverinfo='x+y',
    mode='lines',
    line=dict(width=1.5, color=color_palette['wct_purple']),
    # stackgroup='Eth'
    fill='tozeroy',
))

# High temperature
fig.add_trace(go.Scatter(
    x=df.index, y=Ets_h[2,:]+Ets_c[2,:],
    name='Tmed_s_in_max',
    # hoverinfo='x+y',
    mode='lines',
    line=dict(width=1.5, color=color_palette['plotly_red']),
    fill='tozeroy',
    # stackgroup='Eth'  # define stack group
))


fig.show()